In [3]:
import os
import sys

PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.realpath("__file__")))
PYTHON_CODE_ROOT = os.path.join(PROJECT_ROOT, 'system', 'python')
sys.path.append(PYTHON_CODE_ROOT)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Working with Data
PyTorch has two primitives to work with data: torch.utils.data.DataLoader and torch.utils.data.Dataset. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset.

In [2]:
import torch
from torch.utils.data import DataLoader
import torchvision as tv
import matplotlib.pyplot as plt

PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets. For this tutorial, we will be using a TorchVision dataset.

The torchvision.datasets module contains Dataset objects for many real-world vision data like CIFAR, COCO (full list here). In this tutorial, we use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.

In [7]:
training_data = tv.datasets.FashionMNIST(root=ft.data_location, train=True, download=True, transform=ToTensor())
test_data = tv.datasets.FashionMNIST(root=ft.data_location, train=False, download=True, transform=ToTensor())

We pass the Dataset as an argument to DataLoader. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [18]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


# Creating Models

To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the GPU if available.

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [13]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            torch.nn.Linear(28*28, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512,512),
            torch.nn.ReLU(),
            torch.nn.Linear(512,10),
            torch.nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


# Optimizing the model parameters
To train a model, we need a loss function and an optimizer.

In [14]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # backpropogation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

We also check the model’s performance against the test dataset to ensure it is learning.

In [16]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg_loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [19]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n ---------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
 ---------------
loss: 2.303807 [    0/60000]
loss: 2.299829 [ 6400/60000]
loss: 2.285309 [12800/60000]
loss: 2.280467 [19200/60000]
loss: 2.258211 [25600/60000]
loss: 2.255422 [32000/60000]
loss: 2.244103 [38400/60000]
loss: 2.223840 [44800/60000]
loss: 2.227932 [51200/60000]
loss: 2.239312 [57600/60000]
Test Error: 
 Accuracy: 53.2%, Avg_loss: 0.034615 

Epoch 2
 ---------------
loss: 2.195566 [    0/60000]
loss: 2.195719 [ 6400/60000]
loss: 2.156110 [12800/60000]
loss: 2.190865 [19200/60000]
loss: 2.118849 [25600/60000]
loss: 2.130482 [32000/60000]
loss: 2.111921 [38400/60000]
loss: 2.063315 [44800/60000]
loss: 2.088272 [51200/60000]
loss: 2.100806 [57600/60000]
Test Error: 
 Accuracy: 53.2%, Avg_loss: 0.032013 

Epoch 3
 ---------------
loss: 2.032163 [    0/60000]
loss: 2.018145 [ 6400/60000]
loss: 1.943799 [12800/60000]
loss: 2.011106 [19200/60000]
loss: 1.872233 [25600/60000]
loss: 1.924215 [32000/60000]
loss: 1.868682 [38400/60000]
loss: 1.802293 [44800/60000]
loss: 1.8

# Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [22]:
import os
from common.settings import ac

if not os.path.exists(ac.MODELS_LOCATION_INHOUSE_TORCH + "/python-tutorial-quickstart"):
    os.makedirs(ac.MODELS_LOCATION_INHOUSE_TORCH + "/python-tutorial-quickstart")
torch.save(model.state_dict(), ac.MODELS_LOCATION_INHOUSE_TORCH + "/python-tutorial-quickstart/mlp.pth")

# Loading Models

The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [24]:
model = NeuralNetwork()
model.load_state_dict(torch.load(ac.MODELS_LOCATION_INHOUSE_TORCH + "/python-tutorial-quickstart/mlp.pth"))

<All keys matched successfully>

This model can now be used to make predictions.

In [25]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [27]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Ankle boot, Actual: Ankle boot
